In [ ]:
from utils import rounding
import mathfrom boost_converter import lm3478

In [ ]:
# See https://www.ti.com/lit/ds/symlink/lm3478.pdf section 8

# Design Parameters

V_in_nom = 40 # Nominal input voltage (V)
V_in_min = V_in_nom * 0.95
V_in_max = V_in_nom * 1.1

V_out = 480 # Design output voltage (V)

I_out_min = 1e-3 # Design minimum output current (A)
I_out_max = 100e-3
I_out_nom = 50e-3

f_sw = 250e3 # Switching frequency (Hz)

V_f = 1.2 # Diode nominal forward voltage (V)
V_q = 0.300 # MOSFET on-state voltage drop, estimated (V)

ripple_ratio = 0.35 # Design ripple ratio


In [ ]:
# Duty cycle

D = lm3478.duty_cycle(V_in_nom, V_out, V_q, V_f)
print(f'Design duty cycle (CCM) = {rounding.sig_figs(D*100,4)}%')

In [ ]:
# Inductor sizing

L_CCM = lm3478.min_inductor_ccm(D, V_in_nom, f_sw, I_out_nom)
print(f'Minimum inductor size to support CCM = {rounding.prefix(L_CCM)}H')

I_L = I_out_nom / (1-D) # Average inductor current (A)
I_ripple = I_L * ripple_ratio # Design ripple current (A)
I_L_peak = I_L + I_ripple # Peak inductor current (A)
I_L_valley = I_L - I_ripple # Minimum inductor current (A)
print(f'Inductor average current = {rounding.prefix(I_L)}A')
print(f'Inductor peak current = {rounding.prefix(I_L_peak)}A')
print(f'Inductor valley current = {rounding.prefix(I_L_valley)}A')
print(f'Inductor ripple current = {rounding.prefix(I_ripple)}A')

L_ripple = lm3478.inductor_value_for_ripple(V_in_nom, D, f_sw, I_ripple)
print(f'Minimum inductor size for design ripple ratio ({ripple_ratio*100}%) = {rounding.prefix(L_ripple)}H')

In [ ]:
# Selected inductor value

L = 420e-6 

if L < L_CCM:
    mode = "DCM"
else:
    mode = "CCM"

print(f'Selected L = {rounding.prefix(L)}H')
print(f'Converter will operate in {mode}')

if mode == "CCM":
    I_ripple = D*V_in_nom/(2*f_sw*L) # Actual ripple current
    I_L_peak = I_L + I_ripple # Actual peak current
    I_L_valley = I_L - I_ripple
    
    print(f'I_ripple = {rounding.prefix(I_ripple)}A, I_L_peak = {rounding.prefix(I_L_peak)}A, I_L_valley = {rounding.prefix(I_L_valley)}A')
else:
    D = (1/V_in_nom)*math.sqrt(2*L*(V_out-V_in_nom)*I_out_nom*f_sw)
    I_L_peak = V_in_nom*D/(f_sw*L)
    print(f'Revised duty cycle for DCM = {rounding.sig_figs(D*100,4)}%')
    print(f'I_L_peak = {rounding.prefix(I_L_peak)}A')

In [ ]:
# Feedback resistors
R_f1 = 3.3e6 # Top feedback resistor (Ω)
R_f2 = (1.26*R_f1)/(V_out-1.26) # Bottom feedback resistor
print(f'Feedback resistors (exact): top = {rounding.prefix(R_f1)}Ω, bottom = {rounding.prefix(R_f2)}Ω')

In [ ]:
# Feedback resistor options with E96/E24 values

V_fb = 1.26 # Feedback voltage (V)
I_fb = 1e-4 # Max. current through feedback divider (A)

print('E96 values')
print('Divider V_act %diff I_fb')
resistors = rounding.resistor_divider(V_out, V_fb, I_fb, 96, 5, True)
for t in resistors:
    print(f'{rounding.prefix(t[0])}Ω/{rounding.prefix(t[1])}Ω {rounding.prefix(t[2])}V {rounding.sig_figs(100*t[4],2)}% {rounding.prefix(t[5])}A')

print('\nE24 values')
print('Divider V_act %diff I_fb')
resistors = rounding.resistor_divider(V_out, V_fb, I_fb, 24, 5, True)
for t in resistors:
    print(f'{rounding.prefix(t[0])}Ω/{rounding.prefix(t[1])}Ω {rounding.prefix(t[2])}V {rounding.sig_figs(100*t[4],2)}% {rounding.prefix(t[5])}A')

In [ ]:
# Selected feedback resistors

R_f1 = 12e6
R_f2 = 31.6e3

In [ ]:
# Current sense resistor

V_sense = 156e-3 # Sense voltage from Electrical Characteristics table
ratio_V_sl = 0.49 # V_sl to V_sense ratio
I_sw_limit = 1.2*I_L_peak #(I_out_nom/(1-D)+(D*V_in_nom)/(2*f_sw*L))

R_sen = (V_sense - (D*V_sense*ratio_V_sl))/I_sw_limit

print(f'Design current limit = {rounding.prefix(I_sw_limit)}A')
print(f'R_sen = {rounding.prefix(R_sen)}Ω')

In [ ]:
# Selected sense resistor

R_sen = 48e-3 #rounding.closest_E_series_value(R_sen, 96, 'eq') 
I_sw_limit = (V_sense - (D*V_sense*ratio_V_sl))/R_sen

P_R_sen_avg = I_L*I_L*R_sen 
P_R_sen_peak = I_L_peak*I_L_peak*R_sen
P_R_sen_limit = I_sw_limit*I_sw_limit*R_sen

print(f'Selected R_sen = {rounding.prefix(R_sen)}Ω') 
print(f'I_sw_limit = {rounding.prefix(I_sw_limit)}A') 
print(f'I_L_peak = {rounding.prefix(I_L_peak)}A') 
print(f'Power dissipation: average = {rounding.prefix(P_R_sen_avg)}W; peak = {rounding.prefix(P_R_sen_peak)}W; limit = {rounding.prefix(P_R_sen_limit)}W')

V_sl = 92e-3 # Internal compensation ramp voltage from Electrical Characteristics table 
R_sen_max = (2*V_sl*f_sw*L)/(V_out-2*V_in_nom)

print(f'Maximum R_sen without external compensation = {rounding.prefix(R_sen_max)}Ω')
compensation_required = R_sen < R_sen_max 


In [ ]:
# Compensation

if compensation_required:
    print('External compensation not required')
else:
    print('\nExternal compensation **required**')
    R_sl = (R_sen*(V_out - 2*V_in_nom)/(2*f_sw*L) - V_sl) / 40e-6

    print(f'Minimum R_sl = {rounding.prefix(R_sl)}Ω')

    R_sl_24 = rounding.closest_E_series_value(R_sl, 24, 'gt')
    R_sl_96 = rounding.closest_E_series_value(R_sl, 96, 'gt')

    print(f'Closest values: E24 = {rounding.prefix(R_sl_24)}Ω; E96 = {rounding.prefix(R_sl_96)}Ω')

In [ ]:
# Selected R_sl

R_sl = R_sl_96
V_sl_delta = 40e-6 * R_sl
V_cs = V_sense - D*(V_sl + V_sl_delta)
I_sw_limit = V_cs / R_sen

print(f'Selected R_sl = {rounding.prefix(R_sl)}Ω')
print(f'ΔV_sl = {V_sl_delta}')
print(f'Revised current limit = {rounding.prefix(I_sw_limit)}A')
if I_sw_limit <= 0:
    print(f'Compensation not possible')

R_sen_equation_26 = V_sense/(1.2*I_L_peak+(V_out-V_in_nom)*D/(L*f_sw))
print(f'\nEquation 26 R_sen = {rounding.prefix(R_sen_equation_26)}Ω')
I_sw_limit_equation_26 = (V_sense - (D*V_sense*ratio_V_sl))/R_sen_equation_26
print(f'Equation 26 current limit = {rounding.prefix(I_sw_limit_equation_26)}A')
print('Note: if the equation 26 R_sen value is used in CCM, no external compensation is required.')
print('Note that the equation 26 current limit may be significantly above the design peak current')
print('but this current should not be reached in normal operation.')

In [ ]:
# Diode selection

I_D_peak = lm3478.diode_peak_current(I_L_peak, D, I_out_nom)

print(f'Diode peak current at nominal input/output = {rounding.prefix(I_D_peak)}A')

In [ ]:
# MOSFET

temp_factor = 0.3 # R_ds_on temperature factor to model increase when hot
R_ds_on = 200e-3 # Nominal on resistance at V_gs, q.v. below (Ω)
Q_gs = 8e-9 # Gate-source charge (C)
Q_gd = 10e-9 # Gate-drain charge (C)
V_gs_th = 4 # Threshold voltage (V)
psi_ja = 35 # Junction to ambient thermal resistance (K/W)

# Safety margin multipliers
V_safety_margin = 1.3
I_safety_margin = 1.5
P_safety_margin = 2

# Driver parameters
R_dr_top = 16 # Push-pull driver top switch R_ds_on resistance (Ω)
R_dr_btm = 16 # Push-pull driver bottom switch R_ds_on resistance (Ω)
V_gs = 7.2 # Driver maximum voltage swing (V)

t_LH = 13e-9 # Based on LM3478 datasheet; can also be estimated by (Q_gs/2 + Q_gd) * R_dr_top / (V_gs - V_gs_th)
t_HL = 13e-9 # Can be estimated by (Q_gs/2 + Q_gd) * R_dr_btm / (V_gs - V_gs_th)

P_cond = I_L**2 * R_ds_on * temp_factor * D
P_sw = 0.5 * I_L_peak * V_out * t_LH * f_sw + 0.5 * I_L_valley * V_out * t_HL * f_sw
P_MOSFET = P_cond + P_sw

T_j = 25 + P_MOSFET * psi_ja

print('''
MOSFET parameters
=================''')
print(f'Breakdown V_ds = {rounding.prefix(V_out*V_safety_margin)}V')
print(f'Max. continuous I_D = {rounding.prefix(I_L*I_safety_margin)}A')
print(f'Max. pulsed I_D = {rounding.prefix(I_L_peak*I_safety_margin)}A')

print('''
Power dissipation
-----------------''')
print(f'Conduction loss = {rounding.prefix(P_cond)}W')
print(f'Estimated rise time = {rounding.prefix(t_LH)}s')
print(f'Estimated fall time = {rounding.prefix(t_HL)}s')
print(f'Switching loss (approx.) = {rounding.prefix(P_sw)}W')
print(f'Total power dissipation = {rounding.prefix(P_MOSFET)}W')
print(f'Junction temperature at 25°C = {T_j}°C')
if T_j > 150:
    print('Junction temperature is **outside absolute maximum limits**')

In [ ]:
# Input and output capacitors

I_Cin_RMS = I_ripple/math.sqrt(3)
I_Cout_RMS = math.sqrt((1-D)*((I_out_nom**2 * D/(1-D)) + I_ripple**2 / 3))

print(f'Input capacitor I_RMS = {rounding.prefix(I_Cin_RMS)}A')
print(f'Output capacitor I_RMS = {rounding.prefix(I_Cout_RMS)}A')